# Clustering

In [57]:
import pandas as pd

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, QuantileTransformer, PowerTransformer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from yellowbrick.cluster.elbow import KElbowVisualizer
from yellowbrick.cluster import silhouette_visualizer, intercluster_distance

# Visualization
import plotly.express as px
import plotly.io as pio
pd.options.plotting.backend = "plotly"
pio.templates.default = "seaborn"

In [58]:
df_players = pd.read_csv("./datasets/players.csv", index_col=0)
df_players = df_players[df_players.gender == "f"]

In [59]:
feautures = ["rel_df", "rel_1stIn", "rel_2ndWon", "1WonOn1In", "rel_bpSaved"]
df_data = df_players[feautures].reset_index(drop=True)
# df_data = pd.DataFrame(StandardScaler().fit_transform(df_data), columns=df_data.columns)
#df_data = pd.DataFrame(RobustScaler(unit_variance=True).fit_transform(df_data), columns=df_data.columns)
#df_data = df_data[(np.abs(stats.zscore(df_data)) < 2).all(axis=1)]
df_data = pd.DataFrame(QuantileTransformer().fit_transform(df_data), columns=df_data.columns)
df_data = df_data.round(3)

df_data.boxplot(column=feautures)

/home/reuseman/.miniconda3/envs/dm/lib/python3.9/site-packages/sklearn/preprocessing/_data.py:2588: UserWarning:

n_quantiles (1000) is greater than the total number of samples (714). n_quantiles is set to n_samples.



In [60]:
df_data

,rel_df,rel_1stIn,rel_2ndWon,1WonOn1In,rel_bpSaved
0,0.750,0.234,0.694,0.393,0.087
1,0.295,0.302,0.213,0.886,0.369
2,0.743,0.248,0.850,0.729,0.931
3,0.042,0.526,0.649,0.830,0.849
4,0.620,0.290,0.668,0.816,0.499
...,...,...,...,...,...
709,0.530,0.874,0.168,0.450,0.412
710,0.748,0.285,0.607,0.112,0.942
711,0.310,0.292,0.795,0.576,0.896
712,0.785,0.017,0.227,0.233,0.060


In [140]:
# Create a PCA instance: pca
pca = PCA(n_components=5)
principalComponents = pca.fit_transform(df_data)
# Plot the explained variances
features = range(pca.n_components_)
px.bar(x=features, y=pca.explained_variance_ratio_)

# Save components to a DataFrame
PCA_components = pd.DataFrame(principalComponents)

px.scatter(x=PCA_components[0], y=PCA_components[1])

## K-means

### Find Optimal K

In [ ]:
model = KMeans()
sse_visualizer = KElbowVisualizer(model, k=(2,12), timings=False)
sse_visualizer.fit(df_data)
sse_visualizer.show()

silhouette_visualizer = KElbowVisualizer(model, k=(2,12), timings=False, metric="silhouette")
silhouette_visualizer.fit(df_data)
silhouette_visualizer.show()

calinski_visualizer = KElbowVisualizer(model, k=(2,12), metric='calinski_harabasz', timings=False)
calinski_visualizer.fit(df_data)
calinski_visualizer.show()

In [ ]:
silhouette_visualizer(KMeans(5, random_state=42), df_data, colors='yellowbrick')

In [ ]:
intercluster_distance(kmeans, df_data)

In [61]:
# sse_scores = list()
# silhoutte_scores = list()
# davies_scores = list()
# calinski_harabasz_scores = list()

# max_k = 30
# for k in range(2, max_k + 1):
#     kmeans = KMeans(n_clusters=k, n_init=10, max_iter=100, init="k-means++")
#     kmeans.fit(df_data)    
    
#     # Sum of squared distances of samples to their closest cluster center
#     sse_scores.append(kmeans.inertia_)
#     davies_scores.append(davies_bouldin_score(df_data, kmeans.labels_))
#     silhoutte_scores.append(silhouette_score(df_data, kmeans.labels_))
#     calinski_harabasz_scores.append(calinski_harabasz_score(df_data, kmeans.labels_))

# df = pd.DataFrame({"K": list(range(2, max_k + 1)), "sse": sse_scores, "sil": silhoutte_scores, "davies": davies_scores, "calinski": calinski_harabasz_scores})
# df.plot(x="K", y=["sse"], kind="line").update_traces(mode='lines+markers').show()
# df.plot(x="K", y=["calinski"], kind="line").update_traces(mode='lines+markers').show()
# df.plot(x="K", y=["sil", "davies"], kind="line").update_traces(mode='lines+markers').show()


In [142]:
optimal_k = 4
kmeans = KMeans(n_clusters=optimal_k, n_init=10, max_iter=100, init="k-means++")
kmeans.fit(df_data)
# print("SSE:",sse_scores[optimal_k - 2]," - SILHOUETTE:",silhoutte_scores[optimal_k - 2])

df_players["cluster"] = kmeans.labels_.astype(str)
df_players = df_players.round(3)

### Result analysis

Cluster distribution

In [ ]:
df_players.groupby("cluster").count()["name"].plot.bar()

#### Interpretatation

In [102]:
px.scatter_matrix(df_players,
    dimensions=feautures,
    color="cluster")

In [ ]:

print(f'Most frequent values per cluster')
out_dict = {}
for cluster in range(0, optimal_k):
    temp_df = df_players.groupby(by='cluster').get_group(str(cluster))
    temp_dict = {}

    for col in temp_df.columns:
        temp_dict[col] = temp_df[col].value_counts().idxmax()
    out_dict[cluster] = temp_dict

pd.DataFrame(out_dict)

In [103]:

for feature in df_players.columns.drop(["name"]).to_list():
  px.histogram(df_players, x=feature, facet_col="cluster", color=df_players.gender).show()

In [69]:
pd.set_option('display.max_columns', None)
df_players.groupby("cluster").agg(["mean", "std"])

total_tourneys_played            total_matches_played             \
                         mean        std                 mean        std   
cluster                                                                    
0                   82.269461  39.580699           172.095808  87.193472   
1                   84.757143  37.310325           188.609524  86.659151   
2                   86.942529  37.671797           190.965517  86.895851   
3                   64.171779  39.938780           131.901840  87.970830   

        total_matches_won            matches_won_ratio            \
                     mean        std              mean       std   
cluster                                                            
0               93.275449  51.407264          0.510251  0.113088   
1              107.695238  54.143991          0.556386  0.081807   
2              108.166667  53.514741          0.548316  0.091839   
3               70.687117  52.443800          0.484509  0.138989   

        mean_performance_index           max_performance_index            \
                          mean       std                  mean       std   
cluster                                                                    
0                     0.300156  0.089672              0.786347  0.178742   
1                     0.334019  0.074373              0.842871  0.142042   
2                     0.322897  0.074269              0.813144  0.133672   
3                     0.284926  0.110682              0.728147  0.211129   

        min_performance_index               ht                  age            \
                         mean  std        mean       std       mean       std   
cluster                                                                         
0                         0.0  0.0  170.450000  6.708008  21.988701  4.519976   
1                         0.0  0.0  175.821429  5.665709  21.815867  4.845711   
2                         0.0  0.0  172.369231  5.964921  22.650828  4.485358   
3                         0.0  0.0  172.500000  6.654155  20.912951  4.212326   

        mean_minutes            max_minutes            minutes_entropy  \
                mean        std        mean        std            mean   
cluster                                                                  
0          92.008569  13.667401  156.473054  37.564212        2.869353   
1          94.669281   8.080995  169.033333  32.539322        3.790214   
2          95.975557  10.032858  177.626437  39.577006        3.831891   
3          87.396491  13.323308  141.865031  33.074461        2.430166   

                  mean_rank_points             max_rank_points               \
              std             mean         std            mean          std   
cluster                                                                       
0        1.113891       194.294814  217.364093      384.879988   445.729787   
1        1.313698       597.052900  818.640440     1269.999886  1699.857382   
2        1.410053       643.192057  911.761237     1218.212437  1630.456823   
3        0.889051       138.168693  114.586255      253.901215   192.925444   

        last_rank_points              variance_rank_points                \
                    mean          std                 mean           std   
cluster                                                                    
0             243.790952   302.947040         20748.186281  6.162812e+04   
1             850.213848  1342.937635        306254.573110  1.045453e+06   
2             628.375644   832.582828        227399.207040  6.124427e+05   
3             150.946988   147.096041         11036.497828  1.766812e+04   

        mean_tourney_spectators              max_tourney_spectators  \
                           mean          std                   mean   
cluster                                                               
0                   3569.118653  1049.095111           11590.982036   
1      

## Density-based

## Hierarchical

## Optional

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=56da3ab5-e195-41aa-a609-f5fefeb3379d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>